# This notebook is used to prepare training data from Excel sheets with all training data

In [20]:
from naec import config
from naec import preprocess_audio_data
import pandas as pd
import numpy as np
import h5py

import tensorflow as tf
import tensorflow_io as tfio

In [21]:
NFFT = 512
WINDOW_SIZE_SAMPLES = 512
STRIDE_SIZE_PERC = 0.5
STRIDE_SIZE_SAMPLES = int(WINDOW_SIZE_SAMPLES * STRIDE_SIZE_PERC)

In [22]:
train_data = pd.read_excel("train_data.xlsx")

In [23]:
last_rows = train_data.tail(10)

In [24]:
train_dataset = h5py.File("train_dataset.h5", 'w')

columnNo = 10
dt = h5py.vlen_dtype(np.dtype('float32'))
x_dset = train_dataset.create_dataset('x_dataset', (columnNo,), dtype=dt, compression="gzip", compression_opts=9)
y_dset = train_dataset.create_dataset('y_dataset', (columnNo,), dtype=dt, compression="gzip", compression_opts=9)
print('dt',dt)

dt object


In [25]:
import tensorflow as tf
import tensorflow_io as tfio


def convert_audio_to_spectrogram(filepath, nfft, window_size_samples, stride_size_samples):
    audio = tfio.audio.AudioIOTensor(filepath)
    audio_slice = audio[:]
    audio_tensor = tf.squeeze(audio_slice, axis=[-1])
    if audio_tensor.dtype != tf.float32:
        audio_tensor = tf.cast(audio_tensor, tf.float32) / 32768.0
    position = tfio.audio.trim(audio_tensor, axis=0, epsilon=0.1)
    start = position[0]
    stop = position[1]
    processed = audio_tensor[start:stop]
    spectrogram = tfio.audio.spectrogram(
        processed, nfft=nfft, window=window_size_samples, stride=stride_size_samples)

    spectrogram = tf.math.log(spectrogram).numpy()
    return spectrogram

In [26]:
i = 0
for index, row in last_rows.iterrows():
    x_data_filename = row['DT Mic'].replace('\\', '/')
    x_data_spectrogram = convert_audio_to_spectrogram(x_data_filename, NFFT, WINDOW_SIZE_SAMPLES, STRIDE_SIZE_SAMPLES)

    y_data_filename = row['DT Clean'].replace('\\', '/')
    y_data_spectrogram = convert_audio_to_spectrogram(y_data_filename, NFFT, WINDOW_SIZE_SAMPLES, STRIDE_SIZE_SAMPLES)
    x_dset[i] = x_data_spectrogram.flatten()
    y_dset[i] = y_data_spectrogram.flatten()
    i = i + 1

In [27]:
train_dataset.close()